# Food in Art

In [11]:
import pandas as pd
import os
import time
import concurrent.futures
from typing import List, Dict
import requests
from ratelimit import limits, sleep_and_retry


In [12]:

# Initialize global session for connection pooling
session = requests.Session()
session.headers.update({
    'User-Agent': 'ArtDataBot/1.0 (jipijipijipi@gmail.com) Python/requests',
    'Accept': 'application/json'
})

@sleep_and_retry
@limits(calls=5, period=1)  # Limit to 5 calls per second
def run_sparql_query(query: str, endpoint_url: str) -> Dict:
    """Execute SPARQL query with rate limiting"""
    response = session.get(
        endpoint_url,
        params={'query': query, 'format': 'json'},
        timeout=30
    )
    response.raise_for_status()
    return response.json()

def process_batch(batch_qids: List[str], endpoint_url: str, query_template: str, src_column_name: str, max_retries: int = 5) -> pd.DataFrame:
    """Process a single batch of QIDs with a customizable SPARQL query template."""
    qid_list_str = ' '.join(f'wd:{qid}' for qid in batch_qids)
    # Insert the QID list and src_column_name into the query template
    batch_query = query_template.format(qid_list=qid_list_str, src_column_name=src_column_name)
    
    for retry in range(max_retries):
        try:
            results = run_sparql_query(batch_query, endpoint_url)
            bindings = results['results']['bindings']
            
            if not bindings:
                return pd.DataFrame()

            # Dynamically construct data extraction based on available fields
            data = []
            for b in bindings:
                row = {key: b.get(key, {}).get('value') for key in b}
                data.append(row)
            
            return pd.DataFrame(data)
            
        except Exception as e:
            if retry == max_retries - 1:
                print(f"Max retries exceeded: {e}")
                return pd.DataFrame()
            time.sleep(2 ** retry)  # Exponential backoff
    
    return pd.DataFrame()

def chunk_list(lst: List, chunk_size: int) -> List[List]:
    """Split a list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def get_supplement_from_wikidata(name, src_path, src_column_name, query_template: str):
    # Configuration
    endpoint_url = "https://query.wikidata.org/sparql"
    batch_size = 50
    max_workers = 3  # Adjust based on your needs and API limits
    checkpoint_frequency = 10  # Save checkpoint every N batches

    if os.path.exists(f'data/wikidata_{name}.csv'):
        print("Final file already exists. Skipping data retrieval.")
        return

    # Create checkpoint directory if it doesn't exist
    os.makedirs('data/checkpoints', exist_ok=True)

    # Load checkpoint if exists
    checkpoint_file = f'data/checkpoints/{name}_checkpoint.csv'
    batch_index_file = f'data/checkpoints/{name}_batch_index_checkpoint.txt'
    
    if os.path.exists(checkpoint_file) and os.path.exists(batch_index_file):
        detailed_data = pd.read_csv(checkpoint_file)
        with open(batch_index_file, 'r') as f:
            start_batch = int(f.read())
        print(f"Resuming from batch {start_batch}")
    else:
        detailed_data = pd.DataFrame()
        start_batch = 0

    # Read and prepare author data
    basic_data = pd.read_csv(src_path)
    results = basic_data[[src_column_name]].drop_duplicates().reset_index(drop=True)
    results = results.dropna(subset=[src_column_name])  # Drop rows where 'author_wikidata' is NaN
    # Drop values that do not start with 'http://www.wikidata.org/entity'
    results = results[results[src_column_name].str.startswith('http://www.wikidata.org/entity')]
    # Convert item URIs to Q-IDs and create batches
    src_qids = [uri.split('/')[-1] for uri in results[src_column_name].tolist()]
    
    batches = chunk_list(src_qids, batch_size)

    print(f"Processing {len(batches)} batches with {len(src_qids)} records...")

    # Process batches with ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_batch, batch, endpoint_url, query_template, src_column_name): batch_idx 
            for batch_idx, batch in enumerate(batches[start_batch:], start=start_batch)
        }
        
        completed_batches = 0
        for future in concurrent.futures.as_completed(futures):
            batch_idx = futures[future]
            try:
                batch_df = future.result()
                if not batch_df.empty:
                    detailed_data = pd.concat([detailed_data, batch_df], ignore_index=True)
                
                completed_batches += 1
                print(f"Completed batch {batch_idx + 1}/{len(batches)} "
                      f"({(batch_idx + 1)/len(batches)*100:.1f}%)")

                # Save checkpoint periodically
                if completed_batches % checkpoint_frequency == 0:
                    detailed_data.to_csv(checkpoint_file, index=False)
                    with open(batch_index_file, 'w') as f:
                        f.write(str(batch_idx + 1))
                    print(f"Checkpoint saved at batch {batch_idx + 1}")

            except Exception as e:
                print(f"Error processing batch {batch_idx}: {e}")

    # Final processing
    print("Processing complete. Preparing final dataset...")
    detailed_data.drop_duplicates(inplace=True)
    final_data = pd.merge(results, detailed_data, on=src_column_name, how='left')
    
    # Save final results
    final_data.to_csv(f'data/wikidata_{name}.csv', index=False)
    print(f"Data saved to wikidata_{name}.csv")

    # Clean up checkpoints
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
    if os.path.exists(batch_index_file):
        os.remove(batch_index_file)


author_query_template = """
SELECT DISTINCT ?{src_column_name} ?author_name ?country ?country_label ?gender ?gender_label 
       ?date_of_birth ?place_of_birth ?place_of_birth_label 
       ?place_of_birth_country ?place_of_birth_country_label 
WHERE {{
    VALUES ?{src_column_name} {{ {qid_list} }}
    
    # Author-specific information here
    OPTIONAL {{
        ?{src_column_name} rdfs:label ?author_name.
        FILTER(LANG(?author_name) = "en")
    }}
    OPTIONAL {{
        ?{src_column_name} wdt:P27 ?country.
        ?country rdfs:label ?country_label.
        FILTER(LANG(?country_label) = "en")
    }}
    OPTIONAL {{
        ?{src_column_name} wdt:P21 ?gender.
        ?gender rdfs:label ?gender_label.
        FILTER(LANG(?gender_label) = "en")
    }}
    OPTIONAL {{
        ?{src_column_name} wdt:P569 ?date_of_birth.
    }}
    OPTIONAL {{
        ?{src_column_name} wdt:P19 ?place_of_birth.
        ?place_of_birth rdfs:label ?place_of_birth_label.
        FILTER(LANG(?place_of_birth_label) = "en")
        OPTIONAL {{
            ?place_of_birth wdt:P17 ?place_of_birth_country.
            ?place_of_birth_country rdfs:label ?place_of_birth_country_label.
            FILTER(LANG(?place_of_birth_country_label) = "en")
        }}
    }}
}}
"""
paintings_query_template = """
    SELECT ?item ?creation_date ?origin_country ?display_country ?type ?school ?time_period ?image_url (GROUP_CONCAT(?depicts_label; separator=", ") AS ?depicts) WHERE {{
    VALUES ?item {{ {qid_list} }}
    
    OPTIONAL {{ ?item wdt:P571 ?creation_date. }}
    
    # Origin country of the item or the author
    OPTIONAL {{
        ?item wdt:P495 ?origin_country_wd.
        ?origin_country_wd rdfs:label ?origin_country.
        FILTER(LANG(?origin_country) = "en")
    }}
    OPTIONAL {{
        ?item wdt:P50 ?author.
        ?author wdt:P27 ?author_country_wd.
        ?author_country_wd rdfs:label ?author_country.
        FILTER(LANG(?author_country) = "en")
    }}
    BIND(COALESCE(?origin_country, ?author_country) AS ?origin_country)
    
    # Display country
    OPTIONAL {{
        ?item wdt:P276 ?display_location_wd.
        OPTIONAL {{
            ?display_location_wd wdt:P17 ?display_country_wd.
            ?display_country_wd rdfs:label ?display_country.
            FILTER(LANG(?display_country) = "en")
        }}
    }}
    
    # Type
    OPTIONAL {{
        ?item wdt:P136 ?type_wd.
        ?type_wd rdfs:label ?type.
        FILTER(LANG(?type) = "en")
    }}
    
    # School or tradition
    OPTIONAL {{
        ?item wdt:P135 ?school_wd.
        ?school_wd rdfs:label ?school.
        FILTER(LANG(?school) = "en")
    }}
    
    # Time period
    OPTIONAL {{
        ?item wdt:P2348 ?time_period_wd.
        ?time_period_wd rdfs:label ?time_period.
        FILTER(LANG(?time_period) = "en")
    }}
    
    # Image URL
    OPTIONAL {{
        ?item wdt:P18 ?image_url.
    }}
    
    # Depicts
    OPTIONAL {{
        ?item wdt:P180 ?depicts_wd.
        ?depicts_wd rdfs:label ?depicts_label.
        FILTER(LANG(?depicts_label) = "en")
    }}
    }}
    GROUP BY ?item ?creation_date ?origin_country ?display_country ?type ?school ?time_period ?image_url
"""

locations_query_template = """
                    SELECT ?{src_column_name} ?museum_name ?city ?city_label ?country ?country_label ?founding_date ?museum_type ?museum_type_label ?coordinates ?part_of WHERE  {{
                    VALUES ?{src_column_name} {{ {qid_list} }}
                    
                    OPTIONAL {{
                        ?{src_column_name} wdt:P17 ?country.                             # P17 = country
                        ?country rdfs:label ?country_label.                   # Get the label for country
                        FILTER(LANG(?country_label) = "en")
                    }}
                    
                    OPTIONAL {{
                        ?{src_column_name} wdt:P131 ?city.                           # P131 = city
                        ?city rdfs:label ?city_label.                 # Get the label for location
                        FILTER(LANG(?city_label) = "en")
                    }}
                    
                    OPTIONAL {{
                        ?{src_column_name} wdt:P571 ?founding_date.                      # P571 = founding date
                    }}
                    
                    
                    OPTIONAL {{
                        ?{src_column_name} wdt:P31 ?museum_type.                         # P31 = instance of (museum type)
                        ?{src_column_name} rdfs:label ?museum_type_label.           # Get the label for museum type
                        FILTER(LANG(?museum_type_label) = "en")
                    }}
                    
                    OPTIONAL {{
                        ?{src_column_name} wdt:P625 ?coordinates.                        # P625 = coordinates
                    }}
                    
                    OPTIONAL {{
                        ?{src_column_name} rdfs:label ?museum_name.                      # Get the museum's name
                        FILTER(LANG(?museum_name) = "en")
                    }}
                    
                    OPTIONAL {{
                        ?{src_column_name} wdt:P361 ?part_of.                             # Get the space parents QID
                    }}
                    }}
                    """
      
get_supplement_from_wikidata('painters', 'data/wikidata_paintings_ids_final_2.csv', 'author_wikidata', author_query_template,)
get_supplement_from_wikidata('all_paintings', 'data/wikidata_paintings_ids_final_2.csv', 'item', paintings_query_template,)
get_supplement_from_wikidata('all_locations', 'data/wikidata_paintings_ids_final_2.csv', 'location_wikidata', locations_query_template,)

Final file already exists. Skipping data retrieval.
Final file already exists. Skipping data retrieval.
Processing 206 batches with 10296 records...
Completed batch 3/206 (1.5%)
Completed batch 1/206 (0.5%)
Completed batch 2/206 (1.0%)
Completed batch 4/206 (1.9%)
Completed batch 7/206 (3.4%)
Completed batch 6/206 (2.9%)
Completed batch 5/206 (2.4%)
Completed batch 10/206 (4.9%)
Completed batch 8/206 (3.9%)
Completed batch 9/206 (4.4%)
Checkpoint saved at batch 9
Completed batch 12/206 (5.8%)
Completed batch 13/206 (6.3%)
Completed batch 11/206 (5.3%)
Completed batch 15/206 (7.3%)
Completed batch 16/206 (7.8%)
Completed batch 14/206 (6.8%)
Completed batch 17/206 (8.3%)
Completed batch 19/206 (9.2%)
Completed batch 18/206 (8.7%)
Completed batch 21/206 (10.2%)
Checkpoint saved at batch 21
Completed batch 22/206 (10.7%)
Completed batch 20/206 (9.7%)
Completed batch 23/206 (11.2%)
Completed batch 24/206 (11.7%)
Completed batch 25/206 (12.1%)
Completed batch 27/206 (13.1%)
Completed batch 2

In [ ]:
dg = pd.read_csv('IH-Final-Project/data/checkpoints/download_checkpoint.txt')